# Markov

In [19]:
import random
def markov_prop_simulation(steps):
    
    # 가능한 상태 정의
    states = ['빨강', '파랑', '노랑']
    
    # 전이확률행렬 정의
    transition_matrix={
        '빨강' : {'빨강' : 0.5, '파랑':0.3, '노랑':0.2},
        '파랑' : {'빨강' : 0.2, '파랑':0.6, '노랑':0.2},
        '노랑' : {'빨강' : 0.3, '파랑':0.3, '노랑':0.4},
    }
    
    
    # 초기 상태 무작위 선택
    current_state = random.choice(states)
    sequence = [current_state]
    
    # 상태 전이 반복 수행
    for _ in range(steps):
        next_state=random.choices(
            population=list(transition_matrix[current_state].keys()), # 무작위 추출 후보군
            weights=list(transition_matrix[current_state].values()) # 가중치
        )[0]
        sequence.append(next_state)
        current_state = next_state
        
        
    return sequence

simulation_result = markov_prop_simulation(10)
print("상태전이결과:", simulation_result)

상태전이결과: ['빨강', '빨강', '빨강', '파랑', '빨강', '빨강', '파랑', '빨강', '빨강', '빨강', '빨강']


# Markov Chain

In [12]:
import numpy as np 
import random

def generate_weather_forecast(days):
    
    states=["맑음", "강우"]
    
    transition_matrix = {
        "맑음" : {"맑음":0.6, "강우":0.4},
        "강우" : {"맑음":0.7, "강우":0.3}    
    }
    
    
    # 초기 상태를 무작위로 선택
    current_state = random.choice(states)
    forecast = [current_state]
    
    for _ in range(days -1):
        # 상태 전이 반복 수행
        next_state = np.random.choice(
            states, p=[transition_matrix[current_state]["맑음"], 
                       transition_matrix[current_state]["강우"]]
            )
        forecast.append(next_state)
        current_state=next_state
    return forecast

whether_forecast=generate_weather_forecast(7)
print("7일 후 날씨 예측:", whether_forecast)

7일 후 날씨 예측: ['맑음', '강우', '강우', '맑음', '맑음', '강우', '강우']


# Markov Return

In [ ]:
import numpy as np 

# 상태 와 보상 구조 정의

states=["s", "r1", "r2", "r3", "f"]
rewards = {
    ("s", "r1") : 0.5, ("s", "r2") : 1.5,
    ("r1", "r3") : 1.0, ("r1", "r2") : 1.5,
    ("r2", "r3") : 2.0, ("r3", "f") : 3.0 
}

# 감가율
gamma = 1/2

# 가능한 경로
paths = [
    ["s", "r1", "r3", "f" ], # path 1
    ["s", "r1", "r2", "f" ], # path 2
    ["s", "r2", "r3", "f" ] # path 3
] 

# 반환값 게산 정의 함수
def calculate_return(path):
    total_return = 0
    discount_factor=1 # 초기 감가율
    
    for i in range(len(path)-1):
        state, next_state = path[i], path[i+1]
        reward = rewards.get((state, next_state),0) # (state, next_state) 참조, else 0
        total_return += discount_factor * reward
        discount_factor *= gamma
    
    return total_return

# 모든 경로에 대한 반환값 계산 및 출력
returns = {i+1: calculate_return(path) for i, path in enumerate(paths)} # for index, value in enumerate(iterable) 인덱스 항목을 튜플 평태로 반환
for path_id, value in returns.items():
    print(f"경로 {path_id} 반환값: {value:.4f}")
        

경로 1 반환값: 1.7500
경로 2 반환값: 1.2500
경로 3 반환값: 3.2500


# Markov Decision Process, MDP

MDP 는 MRP(markov reward decision)에서 행동과 정책이 추가된 개념. 

MRP : 시간의 흐름에 따라 상태 전이 확률에 영향을 받으며 자연스럽게 이동
MDP : 타임스텝별로 정책에 따라 행동을 선택하고 상태 전이 확률에 영향을 받아 이동


# MDP 상태가치함수 

 어떤 상태에서 시작했을 때 앞으로 받을 수 있는 보상의 총합의 기대값

In [34]:
import numpy as np 

# 상태와 행동, 감가율 정의
states=["s", "r1", "r2", "r3", "f"]
actions=["a1", "a2"]
gamma=0.5

# 정책 (s,a) : 상태 s에서 a를 선택할 확률
policy = {
    "s" : {"a1" : 0.6, "a2":0.4},
    "r1" : {"a1" : 0.7, "a2":0.3},
    "r2" : {"a1" : 1},
    "r3" : {"a2" : 1},
    
}

# 상태전이확률 p (s,a,s')
transition_probs={
    ("s", "a1") : "r1",
    ("s", "a2") : "r2",
    ("r1", "a1") : "r3",
    ("r1", "a2") : "r2",
    ("r2", "a1") : "r3",
    ("r3", "a2") : "f",
}

# 보상함수 R(s,a)
rewards ={
    ("s", "a1") : 0.5,
    ("s", "a2") : 1.5,
    ("r1", "a1") : 1.0,
    ("r1", "a2") : 1.5,
    ("r2", "a1") : 2.0,
    ("r3", "a2") : 3.0,
}

# 경로 시뮬레이션 기반 기대 반환값 계산
def simulate_episode(start_state="s"):
    state = start_state
    total_return=0
    discount=1.0
    
    # 에피소드 반복 실행
    while state !="f":
        # 현재 상태에서 정책에 따라 행동 선택
        action_prob = policy[state]
        action_list = list(action_prob.keys()) # action_prob 후보군
        prob = list(action_prob.values())
        action = np.random.choice(action_list, p=prob)
        
        # 보상 받기
        reward = rewards.get((state, action),0)
        total_return += discount*reward
        
        #다음 상태로 이동
        next_state = transition_probs.get((state,action), "f")
        state = next_state
        discount*=gamma
        
    return total_return


# 여러번 실행
n_episode=10000
returns=[]

for i in range(n_episode):
    episode_return=simulate_episode()
    returns.append(episode_return)
expected_return = np.mean(returns)

print(f"정책을 따른 기대 반환값(에피소드 평균, 상태가치함수): {expected_return:.4f}")

정책을 따른 기대 반환값(에피소드 평균, 상태가치함수): 2.4287
